-*- coding: utf-8 -*-

"""

Created on Thu Aug 23, 2020

@author: Rohitashwa

@Credits: loren ( https://github.com/jjeffus/gofundme)

"""

In [1]:
# !pip install selenium
# !pip install bs4

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import numpy as np
import pandas as pd

In [3]:
url = 'https://www.gofundme.com/discover'
driver = webdriver.Chrome('C:/Users/rohit/OneDrive/Desktop/IIMa/chromedriver_win32/chromedriver.exe')
driver.get(url)

In [4]:
for elem in driver.find_elements_by_link_text('Show all categories'):
        try:
            elem.click()
            print('Succesful click')
        except:
            print('Unsuccesful click')

In [5]:
source = driver.page_source

driver.close()

source

'<html class="" lang="en"><head><script src="https://connect.facebook.net/en_US/sdk.js?hash=c0c8042ecc3a4bf0d7d771203e610281&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script type="text/javascript" async="" src="//static.chartbeat.com/js/chartbeat.js"></script><script id="facebook-jssdk" src="https://connect.facebook.net/en_US/sdk.js"></script><script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-TDTFTZ"></script><script type="text/javascript" async="" src="//cdn3.optimizely.com/js/geo2.js"></script>\n    <meta content="text/html; charset=utf-8" http-equiv="Content-type">\n    <meta content="ie=edge" http-equiv="x-ua-compatible">\n            <meta name="robots" content="index,follow">\n        <title>Discover Fundraisers on GoFundMe</title>\n\n    \n        <!-- self-referencing canonical -->\n        <link rel="canonical" href="https://www.gofundme.com/discover">\n\n                \n                    <link rel="alternate" hreflang="en-au" href="https://

In [6]:
soup = BeautifulSoup(source, 'lxml')
cat_containers = soup.findAll("div", {"class": "section-categories grid-x small-up-2 medium-up-3 large-up-6"})
categories = cat_containers[0].text #contains category names for this section
categories = categories.splitlines()
categories = list(filter(None, categories))

In [7]:
more_containers = soup.findAll("div", {"class": "section-categories grid-x small-up-2 medium-up-3 large-up-6 js-more-categories"})
more_cats = more_containers[0].text
more_cats = more_cats.splitlines()
more_cats = list(filter(None, more_cats))
all_cats = categories + more_cats # should be len(all_cats) == 18
# all_cats

In [8]:
#make url_categories
categories_urls = list(map(lambda x:{x:'https://www.gofundme.com/discover/{}-fundraiser'.format(x.lower())}, all_cats))
categories_urls

[{'Medical': 'https://www.gofundme.com/discover/medical-fundraiser'},
 {'Memorial': 'https://www.gofundme.com/discover/memorial-fundraiser'},
 {'Emergency': 'https://www.gofundme.com/discover/emergency-fundraiser'},
 {'Nonprofit': 'https://www.gofundme.com/discover/nonprofit-fundraiser'},
 {'Education': 'https://www.gofundme.com/discover/education-fundraiser'},
 {'Animals': 'https://www.gofundme.com/discover/animals-fundraiser'},
 {'Business': 'https://www.gofundme.com/discover/business-fundraiser'},
 {'Community': 'https://www.gofundme.com/discover/community-fundraiser'},
 {'Competition': 'https://www.gofundme.com/discover/competition-fundraiser'},
 {'Creative': 'https://www.gofundme.com/discover/creative-fundraiser'},
 {'Event': 'https://www.gofundme.com/discover/event-fundraiser'},
 {'Faith': 'https://www.gofundme.com/discover/faith-fundraiser'},
 {'Family': 'https://www.gofundme.com/discover/family-fundraiser'},
 {'Newlywed': 'https://www.gofundme.com/discover/newlywed-fundraiser'}

# Process to extract individual gofundme urls from an individual category

In [9]:
mydf = pd.read_csv('data/GFM_url_list.csv', sep = '\t')
headers = ["Url", "Category","Position", "Title", "Location","Amount_Raised", "Goal", "Number_of_Donators", "Length_of_Fundraising", "FB_Shares", "GFM_hearts", "Text"]
mydf = mydf.reindex(columns = headers)
mydf.head()

,Url,Category,Position,Title,Location,Amount_Raised,Goal,Number_of_Donators,Length_of_Fundraising,FB_Shares,GFM_hearts,Text
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.gofundme.com/f/face-and-acne-treat...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.gofundme.com/f/donation-for-uterus...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.gofundme.com/f/ht4vqf-eye-surgery,Medical,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.gofundme.com/f/tqtuw-self-employment,Medical,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def extract_urls_from_categories(url, MoreGFMclicks = 5):
    
    # eg. url = 'https://www.gofundme.com/discover/medical-fundraiser'
    driver = webdriver.Chrome('C:/Users/rohit/OneDrive/Desktop/IIMa/chromedriver_win32/chromedriver.exe')
    driver.get(url)
    
    for i in range(MoreGFMclicks):
        for elem in driver.find_elements_by_link_text('Show More'):
            try:
                elem.click()
                print('Succesful click %s' %(i+1))#make this more useful- say what category it is e.g. url.get_category()
            except:
                print('Unsuccesful click %s' %(i+1))
                
            sleep(5) #longer delay - more succesful
            # 5 seconds to ensure GFM doesn't ban us
            
    source = driver.page_source
        
    driver.close()
    
    soup = BeautifulSoup(source, 'lxml')
    
    containers = soup.findAll("div", {"class": "cell grid-item small-6 medium-4 js-fund-tile"})
    
    temp_url = {}
    i = 1
    
    for container in containers:
        for link in container.find_all('a'):
            temp_url[link.get('href')] = i
            i = i + 1 

    temp_url = {k: ((v // 2) - 1) // 3  for k, v in temp_url.items()} #


    return(temp_url)

# Generate lists of list of URL per category

In [11]:
def list_urls(categoryURLs, moreGFMclicks = 5):
    GFM_urls = []
    for url_pair in categoryURLs:
        category = list(url_pair.keys())[0]
        url = list(url_pair.values())[0]
        print("Currently Scraping: "+url)
        GFM_urls.append([extract_urls_from_categories(url, MoreGFMclicks = moreGFMclicks), category])#get category from categories_urls
    print("All done!")
    return(GFM_urls)

In [12]:
GFM_urls = list_urls(categories_urls,2)
GFM_urls

Currently Scraping: https://www.gofundme.com/discover/medical-fundraiser
Succesful click 2
Currently Scraping: https://www.gofundme.com/discover/memorial-fundraiser
Currently Scraping: https://www.gofundme.com/discover/emergency-fundraiser
Currently Scraping: https://www.gofundme.com/discover/nonprofit-fundraiser
Currently Scraping: https://www.gofundme.com/discover/education-fundraiser
Succesful click 2
Currently Scraping: https://www.gofundme.com/discover/animals-fundraiser
Currently Scraping: https://www.gofundme.com/discover/business-fundraiser
Currently Scraping: https://www.gofundme.com/discover/community-fundraiser
Currently Scraping: https://www.gofundme.com/discover/competition-fundraiser
Currently Scraping: https://www.gofundme.com/discover/creative-fundraiser
Currently Scraping: https://www.gofundme.com/discover/event-fundraiser
Currently Scraping: https://www.gofundme.com/discover/faith-fundraiser
Succesful click 2
Currently Scraping: https://www.gofundme.com/discover/famil

[[{'https://www.gofundme.com/f/money-to-buy-hearing-aids-for-me-as-well-as-others': 0,
   'https://www.gofundme.com/f/face-and-acne-treatment': 0,
   'https://www.gofundme.com/f/donation-for-uterus-cancer-treatment': 0,
   'https://www.gofundme.com/f/ht4vqf-eye-surgery': 1,
   'https://www.gofundme.com/f/tqtuw-self-employment': 1,
   'https://www.gofundme.com/f/official-navajo-nation-covid19-relief-fund': 1,
   'https://www.gofundme.com/f/gokalp039a-adim-olun': 2,
   'https://www.gofundme.com/f/emergenza-covid-cesvi-per-bergamo': 2,
   'https://www.gofundme.com/f/save-nil039s-life': 2,
   'https://www.gofundme.com/f/eric-stevens-care-trust': 3,
   'https://www.gofundme.com/f/foreva-strong': 3,
   'https://www.gofundme.com/f/a-better-life-for-livie': 3,
   'https://www.gofundme.com/f/help-a-front-line-nurse-and-baby-get-proper-care': 4,
   'https://www.gofundme.com/f/realiser-l039impossible-avec-nathan': 4,
   'https://www.gofundme.com/f/officer-shay-mikalonis-fund': 4,
   'https://www.

In [13]:
mydf = pd.DataFrame(columns = ["Url", "Category","Position"])
for cat in GFM_urls:
    
    temp_val = np.array(list(cat[0].values()))
    temp_key = np.array(list(cat[0].keys()))
    temp_category = np.repeat(cat[1], len(cat[0]))
    
    temp_df = pd.DataFrame(columns = ["Url", "Category", "Position"])
    temp_df["Position"] = temp_val
    temp_df["Category"] = temp_category
    temp_df["Url"] = temp_key
    
    mydf = mydf.append(temp_df, ignore_index = True)
mydf

,Url,Category,Position
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0
1,https://www.gofundme.com/f/face-and-acne-treat...,Medical,0
2,https://www.gofundme.com/f/donation-for-uterus...,Medical,0
3,https://www.gofundme.com/f/ht4vqf-eye-surgery,Medical,1
4,https://www.gofundme.com/f/tqtuw-self-employment,Medical,1
...,...,...,...
151,https://www.gofundme.com/f/any-help-muc-apprec...,Wishes,10
152,https://www.gofundme.com/f/celebrating-rajkumar,Wishes,10
153,https://www.gofundme.com/f/conscious-homestead...,Wishes,11
154,https://www.gofundme.com/f/nicks-relief,Wishes,11


In [14]:
#flatten list

mydf.to_csv('data/GFM_url_list.csv', sep='\t')